# Segmenting and Clustering Neighborhoods in Toronto
### Coursera-capstone-project

Week 3

### Part 0 Import all libraries

In [1]:
# install web scrap, geocoder and map visualization library
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.1-py36_0         conda-forge
    certifi:         2019.9.11-py36_0  --> 2019.9.11-py36_0     conda-forge

The following packages will be DOWNGRADED:

    ca-certificates: 2019.10.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

### Part 1: Scrap Toronto neighborhood data from website

scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

In [3]:
# retrieve raw data from Wiki page with Beautiful Soup
data_raw = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data = BeautifulSoup(data_raw, 'html.parser')
#data

In [4]:
# scrap web data
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in data.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # remove the new line char from neighborhood cell

In [5]:
# merge lists into a dataframe
d = {'PostalCode':postalCodeList,
     'Borough':boroughList,
     'Neighborhood': neighborhoodList
    }
df_toronto = pd.DataFrame(d)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Remove not assigned rows**
There are some postal codes which are not belongs to any borough.  
We will remove those rows.

In [6]:
# remove rows that borrough "not assigned"
df_toronto = df_toronto.loc[df_toronto.Borough != 'Not assigned']
df_toronto['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [7]:
# use borough name as neighborhood when neighborhood is "not assigned"
df_toronto.loc[df_toronto['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_toronto.loc[df_toronto['Neighborhood'] == 'Not assigned', 'Borough'].values
df_toronto.head()
#df_toronto['Neighborhood'].unique()


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [11]:
# group neighbors with same postal code and borough into a single row
foo = lambda a: ",".join(a) # create custom aggregator
df_toronto_grouped = df_toronto.groupby(['PostalCode','Borough']).agg({'Neighborhood': foo}).reset_index()
df_toronto_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [12]:
# show dataframe size
df_toronto_grouped.shape

(103, 3)

### Part 2: get geo coordinates and add to the Toronto DataFrame  

**Note:** Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [13]:
# download geo-coordinates from the csv file online
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
geo = pd.read_csv('toronto_coordinates.csv')

In [14]:
print(geo.shape)
geo.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# join 2 dataframes using postal code
df_toronto_geo = pd.merge(df_toronto_grouped, geo, how='left', left_on=['PostalCode'], right_on = ['Postal Code']).drop(['Postal Code'],axis=1)
df_toronto_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
# show dataframe size
print(df_toronto_geo.shape)

(103, 5)


### Part 3: cluster the neighborhoods in Toronto

In [17]:
#get geo coordinates for toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [18]:
# visualize neighborhoods in toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, long, post, borough, neigh in zip(df_toronto_geo['Latitude'], df_toronto_geo['Longitude'], df_toronto_geo['PostalCode'], df_toronto_geo['Borough'], df_toronto_geo['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=8,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [19]:
# reduce dataframe to only incl. boroughs that contains "Toronto" in the name
df_toronto_shortlist = df_toronto_geo[df_toronto_geo['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(df_toronto_shortlist.shape)
df_toronto_shortlist.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [20]:
# visualize the shortlist neighborhood
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, long, post, borough, neigh in zip(df_toronto_shortlist['Latitude'], df_toronto_shortlist['Longitude'], df_toronto_shortlist['PostalCode'], df_toronto_shortlist['Borough'], df_toronto_shortlist['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=8,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)
    
map_toronto

**Using FourSquare API to explore the Boroughs**

In [112]:
# The code was removed by Watson Studio for sharing.

In [23]:
# create a function to get 100 nearby venues within 500m to all the neighborhoods in Toronto Shortlist
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# get venues for the neighborhood
toronto_venues = getNearbyVenues(names=df_toronto_shortlist['Neighborhood'],
                                   latitudes=df_toronto_shortlist['Latitude'],
                                   longitudes=df_toronto_shortlist['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [25]:
# check venues dataframe size
print(toronto_venues.shape)
toronto_venues.head()

(1691, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [28]:
# check how many venues for each neighborhood
toronto_venues.groupby('Neighborhood').count()['Venue']

Neighborhood
Adelaide,King,Richmond                                                                                  100
Berczy Park                                                                                              55
Brockton,Exhibition Place,Parkdale Village                                                               23
Business Reply Mail Processing Centre 969 Eastern                                                        19
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara     15
Cabbagetown,St. James Town                                                                               49
Central Bay Street                                                                                       80
Chinatown,Grange Park,Kensington Market                                                                  94
Christie                                                                                                 16
Church and Well

In [39]:
# how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues['Venue Category'].unique() # notice "Neighborhood is one of the venue category name"

There are 228 uniques categories.


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Pizza Place', 'Juice Bar',
       'Dessert Shop', 'Restaurant', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Spa', 'Diner', 'Grocery Store',
       'Indian Restaurant', 'Caribbean Restaurant', 'Coffee Shop',
       'Bakery', 'American Restaurant', 'Lounge', 'Liquor Store',
       'Burger Joint', 'Gym', 'Fish & Chips Shop', 'Park',
       'Sushi Restaurant', 'Steakhouse', 'Pet Store', 'Burrito Place',
       'Fast Food Restaurant', 'Movie Theater', 'Sandwich Place',
       'Intersection', 'Food & Drink Shop', 'Fish Market', 'Gay Bar',
       'Café', 'Seafood Restaurant', 'Middle Eastern Restaurant',
       'Thai Restaurant', 'Comfort Food Restaurant', 'Cheese Shop',
       'Stationery Store', 'Coworking Space', 'Latin American Restaurant',
       'Bar', 'Ga

In [41]:
# analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe but need to change the name to avoid over-writing the venue category "Neighborhood" column
toronto_onehot['Neighbor'] = toronto_venues['Neighborhood'] 
print (toronto_onehot.shape)

# move neighborhood column to the first column
cols_at_first = ['Neighbor']
toronto_onehot = toronto_onehot[[c for c in cols_at_first if c in toronto_onehot]+[c for c in toronto_onehot if c not in cols_at_first] ]

#print (toronto_onehot.shape)
toronto_onehot.head()

(1691, 229)


,Neighbor,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [42]:
# calculate the frequency of each venue category for each neighborhood
toronto_grouped_freq = toronto_onehot.groupby('Neighbor').mean().reset_index()
print(toronto_grouped_freq.shape)
toronto_grouped_freq.head()

(38, 229)


,Neighbor,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.03,0.0,0.0,0.010000,0.0,0.03,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.010000,0.000000,0.080000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.02,0.0,0.0,0.01,0.0,0.0,0.01,0.000000,0.010000,0.000000,0.020000,0.01,0.000000,0

In [44]:
# show top 5 venue categories for each neighbor
num_top_venues = 5

for hood in toronto_grouped_freq['Neighbor']:
    print("----"+hood+"----")
    temp = toronto_grouped_freq[toronto_grouped_freq['Neighbor'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Sushi Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.04
2              Bakery  0.04
3  Seafood Restaurant  0.04
4        Cocktail Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0                    Café  0.09
1             Coffee Shop  0.09
2          Breakfast Spot  0.09
3            Climbing Gym  0.04
4  Furniture / Home Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2                Park  0.05
3          Comic Shop  0.05
4    Recording Studio  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport S

In [103]:
# sort venue in decending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create a dataframe to show top 10 venue categories for each neighbor
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbor']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbor'] = toronto_grouped_freq['Neighbor']

for ind in np.arange(toronto_grouped_freq.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_freq.iloc[ind, :], num_top_venues)

print (neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 11)


,Neighbor,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Hotel,Cosmetics Shop,Bakery,Restaurant,Asian Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cheese Shop,Farmers Market,Steakhouse,Café,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Shopping Mall
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Restaurant,Italian Restaurant,Stadium,Burrito Place,Sandwich Place,Caribbean Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Bar,Coffee Shop,Boat or Ferry,Boutique,Plane,Airport Terminal
5,"Cabbagetown,St. James Town",Coffee Shop,Café,Pizza Place,Italian Restaurant,Restaurant,Pub,Bakery,Park,Convenience Store,Japanese Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Sandwich Place,Burger Joint,Bakery,Salad Place,Spa,Bubble Tea Shop
7,"Chinatown,Grange Park,Kensington Market",Café,Vietnamese Restaurant,Bar,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Bakery,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Dessert Shop
8,Christie,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Candy Store,Italian Restaurant,Diner,Baby Store,Nightclub
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Italian Restaurant,Hotel,Gastropub,Gym,Café


In [104]:
# set number of clusters and cluster the neighbor
kclusters = 4

toronto_grouped_clustering = toronto_grouped_freq.drop('Neighbor', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels back to top 10 venue category dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [105]:
# merge geocode and cluster label
neighborhoods_venues_sorted.groupby(['Cluster Labels']).count()
df_toronto_merged = df_toronto_shortlist

# merge toronto_shortlist with toronto_data to add latitude/longitude for each neighborhood
df_toronto_merged = df_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbor'), on='Neighborhood')

print (df_toronto_merged.shape)
df_toronto_merged.head() # check the last columns!

(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Brewery,Bubble Tea Shop,Caribbean Restaurant,Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Burrito Place,Fish & Chips Shop,Steakhouse,Food & Drink Shop,Intersection,Sushi Restaurant,Burger Joint,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Ice Cream Shop,Fish Market,Middle Eastern Restaurant,Brewery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto_merged['Latitude'], df_toronto_merged['Longitude'], df_toronto_merged['Neighborhood'], df_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters)
       
map_clusters

In [107]:
# exam each cluster 0
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 0, df_toronto_merged.columns[[2] + list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Health Food Store,Pub,Trail,Neighborhood,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,"The Danforth West,Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Brewery,Bubble Tea Shop,Caribbean Restaurant,Restaurant
2,"The Beaches West,India Bazaar",0,Park,Sandwich Place,Burrito Place,Fish & Chips Shop,Steakhouse,Food & Drink Shop,Intersection,Sushi Restaurant,Burger Joint,Ice Cream Shop
3,Studio District,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Ice Cream Shop,Fish Market,Middle Eastern Restaurant,Brewery
5,Davisville North,0,Breakfast Spot,Park,Clothing Store,Food & Drink Shop,Hotel,Gym,Sandwich Place,Falafel Restaurant,Event Space,Ethiopian Restaurant
6,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Bagel Shop,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
7,Davisville,0,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym,Restaurant,Movie Theater
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,Pub,Coffee Shop,Pizza Place,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Liquor Store,Bagel Shop
11,"Cabbagetown,St. James Town",0,Coffee Shop,Café,Pizza Place,Italian Restaurant,Restaurant,Pub,Bakery,Park,Convenience Store,Japanese Restaurant
12,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Italian Restaurant,Hotel,Gastropub,Gym,Café


In [108]:
# exam each cluster 1
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 1, df_toronto_merged.columns[[2] + list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,1,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [109]:
# exam each cluster 2
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 2, df_toronto_merged.columns[[2] + list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,2,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
10,Rosedale,2,Park,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [110]:
# exam each cluster 3
df_toronto_merged.loc[df_toronto_merged['Cluster Labels'] == 3, df_toronto_merged.columns[[2] + list(range(5, df_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park,Summerhill East",3,Gym,Yoga Studio,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


**observation of clusters as follow:**  
- Cluster 0: most related to food
- Cluster 1: most related to garden
- Cluster 2: most related to park
- Cluster 3: most related to exercise